# 1. Install and Import Dependencies

In [ ]:
!pip3 install torch==1.8.2 torchvision==0.9.2 torchaudio==0.8.2 --extra-index-url https://download.pytorch.org/whl/lts/1.8/cu111

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
!cd yolov5 && pip install -r requirements.txt

In [ ]:
import torch
import numpy as np
import cv2
import os
import uuid
import time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
# cropping_info = top, bottom, left, right,
def crop(read_img, top, bottom, left, right):
  height, width, _ = read_img.shape
  cropped = read_img[top:height-bottom, left:width-right]
  return cropped

def autoCropToSquare(read_img):
  height, width, _ = read_img.shape
  if width == height:
    return read_img
  top = 0
  bottom = 0
  left = 0
  right = 0
  if width > height:
    half_excess = (width - height) / 2
    if half_excess % 2 == 0:
      left = right = int(half_excess)
    else:
      right = int(half_excess)
      left = right + 1
  else:
    half_excess = (height - width) / 2
    if half_excess % 2 == 0:
       top = bottom = int(half_excess)
    else:
      bottom = int(half_excess)
      top =  bottom + 1
  cropping_info = top, bottom, left, right,
  return crop(read_img, *cropping_info)

def preprocess(img, pixels, forceSq = False):
  read_img = cv2.imread(img)
  if forceSq:
    cropped = autoCropToSquare(read_img)
  else:
    cropped = read_img
  resized = cv2.resize(cropped, (pixels, pixels))
  cv2.imwrite(img, resized)
  return

def bulkprocess(folder_path, pixels):
  files = os.listdir(folder_path)
  for file in files:
    if file.endswith('.jpg'):
      file_path = os.path.join(folder_path, file)
      print(file_path)
      preprocess(file_path, pixels)
  return

# target = os.path.join('/content/data/images')
# bulkprocess(target, 128)

# 2. Load Model

In [ ]:
model_name = 'yolov5l'
model = torch.hub.load('ultralytics/yolov5', model_name)

# 3. Make Detections with Images

In [ ]:
data_dir = os.path.join('data')
images_dir = os.path.join(data_dir, 'images')
labels_dir = os.path.join(data_dir, 'labels')
captures_dir = os.path.join(data_dir, 'captures')

os.makedirs(images_dir, exist_ok = True)
os.makedirs(labels_dir, exist_ok = True)
os.makedirs(captures_dir, exist_ok = True)

In [ ]:
img = os.path.join(captures_dir, 'img-' + '' + '.jpg')
img = os.path.join(captures_dir, 'truck_1.jpg')

In [ ]:
# preprocess(img, 300)
results = model(img)
results.print()

In [ ]:
%matplotlib inline
plt.imshow(np.squeeze(results.render()))
plt.axis(False)
plt.show()

# 4. Detections through Webcam

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
img_name =  'img-' + str(uuid.uuid1()) + '.jpg'
img_path = os.path.join(captures_dir, img_name)
saved_img = ''
try:
  filename = take_photo(img_path, 1)
  print('Saved to {}'.format(filename))
  saved_img = filename
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
if torch.cuda.is_available():
  print('GPU accessed!')
else:
  print('No GPU!')

In [ ]:
!git clone https://github.com/tzutalin/labelImg

In [ ]:
!pip install pyqt5 lxml --upgrade
!cd labelImg && pyrcc5 -o libs/resources.py resources.qrc

In [ ]:
!cd yolov5 && python train.py --img 300 --batch 16 --epochs 1000 --data custom_dataset.yml --weights yolov5l.pt --workers 2

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp/weights/last.pt', force_reload=True)

In [ ]:
results = model(img)

In [ ]:
%matplotlib inline 
plt.imshow(np.squeeze(results.render()))
plt.axis(False)
plt.show()

In [ ]:
%matplotlib inline
def display(image):
  img = mpimg.imread(image)
  imgplot = plt.imshow(img)
  plt.axis(False)
  plt.show()

In [ ]:
display(img)